In [1]:
#### Dependencies

from PIL import Image
from skimage.feature import greycomatrix, greycoprops

import glob
import math
import matplotlib.pyplot as plt
import os
import random


In [2]:
#### Global variables

model = {}
test_model = {}
map_8bit_to_3bit = [i // 32 for i in range(256)]


In [25]:
#### Functions

def load_img(img_path):
    return Image.open(img_path).convert('L')

def get_img_size(img):
    return img.size

def print_img(img):
    plt.imshow(img, cmap='gray')
    
def get_resized_img(img, dimension):
    return img.resize(dimension)
    
def get_img_colors(img):
    return list(img.getdata())

def get_3bit_img_colors(img):
    img_colors = get_img_colors(img)
    
    loop_count = 0
    for img_color in img_colors:
        img_colors[loop_count] = map_8bit_to_3bit[img_color]
        
        loop_count += 1
        
    return img_colors

def get_img_matrix(img_colors):
    img_matrix = []
    
    loop_count = 0
    img_square_dimension = int(math.sqrt(len(img_colors)))
    
    for row in range(img_square_dimension):
        temp_row = []
        for col in range(img_square_dimension):
            temp_row.append(img_colors[loop_count])
            
            loop_count += 1
        img_matrix.append(temp_row)
        
    return img_matrix

def get_img_features(img):
    img_3bit_colors = get_3bit_img_colors(img)
    img_matrix = get_img_matrix(img_3bit_colors)
    
    glcm_matrix = greycomatrix(img_matrix, distances=[1], angles=[0], levels=12, symmetric=False, normed=False)
    glcm_components = ['contrast', 'correlation', 'energy', 'homogeneity', 'ASM', 'dissimilarity']
    
    img_features = []
    for glcm_component in glcm_components:
        img_features.append(greycoprops(glcm_matrix, glcm_component)[0][0])
        
    return tuple(img_features)

# Modelling

def init_model():
    global model
    global test_model
    
    model = {}
    test_model = {}
    
def get_class_names(training_path):
    return os.listdir(training_path)

def get_class_names_in_model():
    class_names = [class_name for class_name in model]
    
    return class_names

def add_class_names_to_model(class_names):
    global model
    global test_model
    
    for class_name in class_names:
        if(not class_name in model):
            model[class_name] = []
        if(not class_name in test_model):
            test_model[class_name] = []
            
def insert_img_features_into_model(class_name, img_features):
    global model
    
    if(class_name in model):
        model[class_name].append(img_features)
        
def insert_img_features_into_test_model(class_name, img_features):
    global test_model
    
    if(class_name in test_model):
        test_model[class_name].append(img_features)

def load_preprocessed_img(img_path, dimension=(128, 128)):
    img = load_img(img_path)
    img = get_resized_img(img, dimension)
    
    return img
        
def train(training_path, training_rate=0.8, img_type='*.jpg'):
    print('Training...')
    class_names = get_class_names_in_model()
    for class_name in class_names:
        img_paths = glob.glob(training_path + class_name + '/' + img_type)
        
        for img_path in img_paths:
            random_splitter = random.uniform(0, 1)
            
            img = load_preprocessed_img(img_path)
            img_features = get_img_features(img)
            if(random_splitter <= training_rate):
                insert_img_features_into_model(class_name, img_features)
            else:
                insert_img_features_into_test_model(class_name, img_features)
            
    print('--> Done')
    
def get_euclidian_distance(model_img_features, img_features):
    distance = 0.0
    
    for glcm_index in range(6):
        distance += (model_img_features[glcm_index] - img_features[glcm_index]) ** 2
        
    return math.sqrt(distance)

def get_img_features_class(img_features, k_neighbors=1):
    minimum_model_img_distances = [-1 for i in range(k_neighbors)]
    minimum_model_img_class_names = ['unknown' for i in range(k_neighbors)]
    is_first_loop = True
    for class_name in model:
        for model_img_features in model[class_name]:
            euclidian_distance = get_euclidian_distance(model_img_features, img_features)
            
            if(is_first_loop):
                minimum_model_img_distances[0] = euclidian_distance
                minimum_model_img_class_names[0] = class_name
                is_first_loop = False
            else:
                for i in range(k_neighbors):
                    if(euclidian_distance < minimum_model_img_distances[i] or minimum_model_img_distances[i] == -1):
                        for j in range(k_neighbors - 1, i, -1):
                            minimum_model_img_distances[j] = minimum_model_img_distances[j-1]
                            minimum_model_img_class_names[j] = minimum_model_img_class_names[j-1]
                        minimum_model_img_distances[i] = euclidian_distance
                        minimum_model_img_class_names[i] = class_name
                        break
    
    minimum_model_img_class_names = list(filter(lambda val: val != 'unknown', minimum_model_img_class_names))
    img_class_name = max(set(minimum_model_img_class_names), key=minimum_model_img_class_names.count)
    
    return img_class_name
    
def get_img_class(img, k_neighbors=1):
    img_features = get_img_features(img)
    img_class_name = get_img_features_class(img_features, k_neighbors)
    
    return img_class_name
    
def test(k_neighbors=1):
    print('Testing...')
    class_names = get_class_names_in_model()
    total_correct_answer = 0
    total_guess = 0
    
    for class_name in test_model:
        for test_img_features in test_model[class_name]:
            test_img_class_name = get_img_features_class(test_img_features, k_neighbors)
            
            if(class_name == test_img_class_name):
                total_correct_answer += 1
                
            total_guess += 1
    
    accuracy = (total_correct_answer / total_guess) * 100
    
    print('--> Done, accuracy = ' + str(accuracy) + ' %')
    
    return accuracy
    
def validate(validation_path, img_type='*.jpg', k_neighbors=1):
    print('Validating...')
    class_names = get_class_names_in_model()
    total_correct_answer = 0
    total_guess = 0
    
    for class_name in class_names:
        img_paths = glob.glob(validation_path + class_name + '/' + img_type)
        for img_path in img_paths:
            img = load_preprocessed_img(img_path)
            img_class_name = get_img_class(img, k_neighbors)
            
            if(class_name == img_class_name):
                total_correct_answer += 1
                
            total_guess += 1
            
    print('--> Done')
    
    accuracy = (total_correct_answer / total_guess) * 100
    
    return accuracy
    

In [26]:
#### Main

# Defining paths
root_path       = './'
training_path   = root_path + 'training/'
validation_path = root_path + 'validation/'
test_path       = root_path + 'test/'

# Props
epochs = 5
training_rate = 0.8
img_type = '*.jpg'
k_neighbors = 3

# Init
init_model()
class_names = get_class_names(training_path)
add_class_names_to_model(class_names)

for epoch in range(epochs):
    print('Epoch ' + str(epoch + 1))
    train(training_path, training_rate, img_type)
    test(k_neighbors)
    print()

accuracy = validate(validation_path, img_type, k_neighbors)

print('Accuracy: ' + str(accuracy) + ' %')

Epoch 1
Training...
--> Done
Testing...
--> Done, accuracy = 90.72164948453609 %

Epoch 2
Training...
--> Done
Testing...
--> Done, accuracy = 95.60975609756098 %

Epoch 3
Training...
--> Done
Testing...
--> Done, accuracy = 99.37888198757764 %

Epoch 4
Training...
--> Done
Testing...
--> Done, accuracy = 100.0 %

Epoch 5
Training...
--> Done
Testing...
--> Done, accuracy = 100.0 %

Validating...
--> Done
Accuracy: 90.234375 %


In [27]:
img = load_img(test_path + 'cap.jpg')
print(get_img_class(img, 3))
# test correct

cap


In [28]:
img = load_img(test_path + 'cap2.jpg')
print(get_img_class(img, 3))
# test false

tulis


In [29]:
img = load_img(test_path + 'tulis.jpg')
print(get_img_class(img, 3))
# test false

cap
